In [129]:
from sympy import diff, symbols, exp
from sympy.abc import t
import sympy

import numpy as np
import matplotlib as plt

from scipy.optimize import fsolve
from dataclasses import dataclass, field

 ## State Space Model:

$$\frac{dC_A}{dt}=\frac{F}{V}\left(C_{A0}-C_A\right)-k_1(T)C_A\left(C_A+C_B-C_{A0}\right)$$

$$\frac{dC_S}{dt}=\frac{F}{V}\left(C_{S0}-C_S\right)-k_2(T)C_S$$

$$\frac{dT}{dt}=\frac{F}{V}\left(T_0-T\right)+\frac{-\Delta H_1k_1(T)C_A\left(C_A+C_B-C_{A0}\right)-\Delta H_2k_2(T)C_S}{\rho C_p}-\frac{UA\left(T-T_c\right)}{V\rho C_p}$$

In [114]:
F, V, C_A0, C_A, k_1, T, C_A, C_B, k_10, R, E_1, C_S0, C_S, E_2, k_20, H_1, H_2, T_0, T_c, UA, rho, C_p = symbols('F, V, C_A0, C_A, k_1, T, C_A, C_B, k_10, R, E_1, C_S0, C_S, E_2, k_20, H_1, H_2, T_0, T_c, UA, rho, C_p')

C_As, C_Bs, C_Cs, C_Ss, T_s = symbols('C_As, C_Bs, C_Cs, C_Ss, T_s')

k_1 = k_10 * exp(-E_1 / T / R)
k_2 = k_20 * exp(-E_1 / T / R)

f_C_A = F / V * (C_A0 - C_A) - k_1 * C_A * (C_A + C_B - C_A0)

f_C_S = F / V * (C_S0 - C_S) - k_2 * C_S

f_T = F / V * (T_0 - T) + (-H_1 * k_1 * C_A * (C_A + C_B - C_A0) - H_2 * k_2 * C_S) / (rho * C_p) - (UA * (T - T_c)) / (V * rho * C_p)

In [137]:
@dataclass
class SystemParameters():
    k_10: float
    E_1: float 
    k_20: float
    E_2: float
    H_1: float
    H_2: float
    rho: float
    C_p: float
    A: float
    T_c: float
    V: float

@dataclass
class InletConditions():
    F_A0: float
    F_S0: float
    F_B0: float
    v_0: float = field(init=False)
    C_A0: float = field(init=False)
    C_B0: float = field(init=False)
    C_S0: float = field(init=False)
    F: float = field(init=False)

    def __post_init__(self):
        self.F = self.F_A0 + self.F_B0 + self.F_S0

@dataclass
class SteadyState():
    U_s: float
    T_0s: float
    T_s: float = field(init=False)

System_Params = SystemParameters(**{
    "k_10": 4e14,
    "E_1": 1.28e5,  
    "k_20": 1e84,
    "E_2": 8e5,
    "H_1": -45400,
    "H_2":-3.2e5,
    "rho": 12.4,
    "C_p": 254,
    "A": 5.3,
    "T_c": 373,
    "V": 4000,
})

Inlet_Conds = InletConditions(**{
    "F_A0": 1075,
    "F_S0": 750,
    "F_B0": 1275,
})

Inlet_Conds.v_0 = Inlet_Conds.F / System_Params.rho
Inlet_Conds.C_A0 = Inlet_Conds.F_A0 / Inlet_Conds.v_0
Inlet_Conds.C_B0 = Inlet_Conds.F_B0 / Inlet_Conds.v_0
Inlet_Conds.C_S0 = Inlet_Conds.F_S0 / Inlet_Conds.v_0

Steady_States = SteadyState(**{
    "U_s": 14.656e3,
    "T_0s": 410,
})

InletConditions(F_A0=1075, F_S0=750, F_B0=1275, v_0=250.0, C_A0=4.3, C_B0=5.1, C_S0=3.0, F=3100)

In [138]:
f_C_A_subbed = f_C_A.subs({
    C_A0: Inlet_Conds.C_A0,
    k_10: System_Params.k_10,
    R: 8.314,
    F: Inlet_Conds.F,
    V: System_Params.V,
    E_1: System_Params.E_1,
})

f_C_S_subbed = f_C_S.subs({
    C_A0: Inlet_Conds.C_A0,
    k_10: System_Params.k_10,
    R: 8.314,
    F: Inlet_Conds.F,
    V: System_Params.V,
    E_1: System_Params.E_1,
    C_S0: Inlet_Conds.C_S0,
    k_20: System_Params.k_20,
})

f_T_subbed = f_T.subs({
    C_A0: Inlet_Conds.C_A0,
    k_10: System_Params.k_10,
    R: 8.314,
    F: Inlet_Conds.F,
    V: System_Params.V,
    E_1: System_Params.E_1,
    C_S0: Inlet_Conds.C_S0,
    k_20: System_Params.k_20,
    T_0: Steady_States.T_0s,
    UA: Steady_States.U_s * System_Params.A,
    T_c: System_Params.T_c,
    H_1: System_Params.H_1,
    H_2: System_Params.H_2,
    C_p: System_Params.C_p, 
    rho: System_Params.rho,
})

In [118]:
def steady_state_system(x):
    '''
    Missing something for C_B
    '''
    f = x*0

    _C_A = x[0]
    _C_S = x[1]
    _T = x[2]

    f[0] = f_C_A_subbed.subs({C_A: _C_A, C_S: _C_S, T: _T})
    f[1] = f_C_S_subbed.subs({C_A: _C_A, C_S: _C_S, T: _T})
    f[2] = f_T_subbed.subs({C_A: _C_A, C_S: _C_S, T: _T})

    return f

# x = np.array([1, 1, 1])

# _C_A = x[0]
# _C_S = x[1]
# _T = x[2]

# f_C_A_subbed.subs({C_A: _C_A, C_S: _C_S, T: _T})
# f_C_S_subbed.subs({C_A: _C_A, C_S: _C_S, T: _T})

### Linearization

In [119]:
A = sympy.matrices.Matrix([
    [f_C_A.diff(C_A), f_C_A.diff(C_S), f_C_A.diff(T)],
    [f_C_S.diff(C_A), f_C_S.diff(C_S), f_C_S.diff(T)],
    [f_T.diff(C_A), f_T.diff(C_S), f_T.diff(T)],
])

b = sympy.matrices.Matrix([
    [f_C_A.diff(UA)],
    [f_C_S.diff(UA)],
    [f_T.diff(UA)],
])

c = sympy.matrices.Matrix([
    [0, 0, 1],
])

d = 0

state_vector = sympy.matrices.Matrix([
    [C_A],
    [C_S],
    [T],
])